# This notebook is for testing purposes , It is not the final deliverable.
## Contents:
0. [Message Encoding with message blocks](#0)
1. [Diffie-Hellman Key Exchange](#1)
2. [AlGamal](#2)
3. [Initilization](#3)
4. [Handling Messages](#4)
5. [Integration to chat.py](#5)

0. [Message Encoding with message blocks](#0)

### Import relevant libraries

In [11]:
from helpers import dh_gamal
from helpers import utils

### Message Encoding with message blocks

The main reason to use Base64 is to be trasmit data over a channel that doesn't allow binary data transfer. That said, now it should be clear that you can have a stream encoded in Base64 that rapresent a stream UTF8 encoded.

[What is the difference between utf-8 and base64](https://stackoverflow.com/questions/3866316/whats-the-difference-between-utf8-utf16-and-base64-in-terms-of-encoding#:~:text=But%20in%20languages%20where%20there's,to%20keep%20the%20concerns%20separate.&text=%2B1%20Base64%20is%20a%20way,ways%20to%20encode%20Unicode%20text.)

#### Trying different encodings

In [2]:
message = "hello world hello baba hello mama hello teta hello geddo"

In [ ]:
print("Original Message: " , message)

print("#############################################################")
print("UTF-ENCDOING")
print("#############################################################")
print("Message with UTF-8 encoding: ",message.encode('utf-8'))
print("Message with UTF-16 encoding: ",message.encode('utf-16'))
print("Message with UTF-32 encoding: ",message.encode('utf-32'))


print("#############################################################")
print("INT.FROM_BYTES little endian encoding")
print("#############################################################")
# testing int.from_bytes
print(int.from_bytes(message.encode('utf-8'), 'little'))
print(int.from_bytes(message.encode('utf-16'), 'little'))
print(int.from_bytes(message.encode('utf-32'), 'little'))


print("#############################################################")
print("Base64 Encoding")
print("#############################################################")
# encode with base64
message_bytes = message.encode('utf-8')
print("Message with base64 encoding: ", base64.b64encode(message_bytes))
print("Message with a85 encoding: ", base64.a85encode(message_bytes))
print("Message with b85 encoding: ", base64.b85encode(message_bytes))
print("Message with b16 encoding: ", base64.b16encode(message_bytes))
print("Message with b32 encoding: ", base64.b32encode(message_bytes))
print("Message with b85 encoding: ", base64.b85encode(message_bytes))
print("Message with b32hex encoding: ", base64.b32hexencode(message_bytes))
print("Message with standard base64 encoding: ", base64.standard_b64encode(message_bytes))
print("Message with urlsafe base64 encoding: ", base64.urlsafe_b64encode(message_bytes))

print("#############################################################")
print("INT FROM_BYTES little endian encoding")
print("#############################################################")
# testing int.from_bytes
print(int.from_bytes(base64.b64encode(message_bytes), 'little'))
print(int.from_bytes(base64.a85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b16encode(message_bytes), 'little'))
print(int.from_bytes(base64.b32encode(message_bytes), 'little'))
print(int.from_bytes(base64.b85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b32hexencode(message_bytes), 'little'))
print(int.from_bytes(base64.standard_b64encode(message_bytes), 'little'))
print(int.from_bytes(base64.urlsafe_b64encode(message_bytes), 'little'))



#### decoding (here we choose either utf-8 or base85)

In [ ]:
print(message.encode('utf-8').decode('utf-8'))
print(base64.b85decode(base64.b85encode(message.encode('utf-8'))).decode('utf-8'))

** Using Message Blocks **

In [ ]:
message_blocks = [message[i:i+4] for i in range(0, len(message), 4)]

# trying utf-8 and base85 encoding
utf_8_blocks = [block.encode('utf-8') for block in message_blocks]
print(utf_8_blocks)
base85_blocks = [base64.b85encode(block) for block in utf_8_blocks]
print(base85_blocks)
int_blocks = [int.from_bytes(block,'little') for block in base85_blocks]
print(int_blocks)
int_from_utf = [int.from_bytes(block,'little') for block in utf_8_blocks]
print(int_from_utf)


In [3]:
# we need large two prime numbers 
def is_prime(n):
    if n <= 1 :
            return False
    else:
            return all([n%i!=0 for i in range(2,int(n/2)+1)])

@cache
def generate_prime(low,high,how_many):
    counter = how_many
    primes = []
    while counter :
            prime_num = 1
            while not is_prime(prime_num):
                    prime_num = np.random.randint(low=low,high=high)
            counter -= 1
            primes.append(prime_num)
    return primes

primes = generate_prime(1000,5000,2)
print(primes)

[4909, 2903]


In [12]:
def enforce_message_encoding(message:str , n:int , encoding:str = 'utf-8') -> Union[int,List[int]]:
    # try encoding the whole message 
    message_bytes = message.encode('utf-8')
    if int.from_bytes(message_bytes,'little') > n :
    # split the message into blocks and encode each block
        message_blocks = [message[i:i+2] for i in range(0, len(message), 2)]
        utf_8_blocks = [block.encode('utf-8') for block in message_blocks]
        if encoding == 'base85':
            utf_8_blocks = [base64.b85encode(block) for block in utf_8_blocks]
        int_blocks = [int.from_bytes(block,'little') for block in utf_8_blocks]
    else:
        if encoding == 'base85':
            message_bytes = base64.b85encode(message_bytes)
        int_blocks = int.from_bytes(message_bytes,'little')
    return int_blocks


In [16]:
print(enforce_message_encoding('hello baba',primes[0]*primes[1],'utf-8'))
print(enforce_message_encoding('hello baba',primes[0]*primes[1],'base85'))

[25960, 27756, 8303, 24930, 24930]
[6318936, 8138073, 2783578, 7369046, 7369046]


### 1. [Diffie-Hellman Key Exchange](#1)

In [18]:
def deffie_hellman(prime:int , generator:int , private_key:int):
    # generator is the primitive root of the prime number
    # private_key is the private key of the user
    # public key is the generator raised to the power of the private key modulo the prime number
    public_key = (generator
                    **private_key) % prime
    return public_key

def secret_key_from_dh(prime:int , public_key:int , private_key:int):
    # the secret key is the public key raised to the power of the private key modulo the prime number
    secret_key = (public_key
                    **private_key) % prime
    return secret_key

# testing 
prime = 23
generator = 5
alice_private_key = 6
bob_private_key = 15
alice_public_key = deffie_hellman(prime,generator,alice_private_key)
bob_public_key = deffie_hellman(prime,generator,bob_private_key)
print(alice_public_key)
print(bob_public_key)
alice_secret_key = secret_key_from_dh(prime,bob_public_key,alice_private_key)
bob_secret_key = secret_key_from_dh(prime,alice_public_key,bob_private_key)
print(alice_secret_key)
print(bob_secret_key)


8
19
2
2


In [9]:
with open('agree.txt','r') as f:
    sender_keys = f.readlines()
    for key in sender_keys:
        if 'q_dh' in key:
            q_dh = int(key.split('=')[-1])
        elif 'a_dh' in key:
            a_dh = int(key.split('=')[-1])

print(q_dh,a_dh)


353 3


### 2. [AlGamal](#2)

### 3. [Initilization](#3)

### 4. [Handling Messages](#4)

### 5. [Integration to chat.py](#5)